In [7]:
from langchain_openai import ChatOpenAI
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
import os
import time

In [9]:
load_dotenv()

True

# Components

In [10]:
llm = ChatOpenAI(
    openai_api_base="https://api.xiaoai.plus/v1",
    openai_api_key= os.getenv('OPENAI_API_KEY'),
    temperature=0.3
)
res = llm.invoke("What is after Monday?")
print(res.content)

After Monday comes Tuesday.


In [11]:
messages = [
    SystemMessage(content = "You are an expert data scientist"),
    HumanMessage(content = "Write a very simple python script that trains a neural network on simulated data")
]
response = llm(messages)
print(response.content)

/opt/anaconda3/envs/st_lc/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Sure! Here's a simple Python script that trains a neural network on simulated data using the Keras library:

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Generate simulated data
np.random.seed(0)
X = np.random.rand(100, 2)
y = np.random.randint(2, size=100)

# Define the model architecture
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=10)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f'Loss: {loss}, Accuracy: {accuracy}')
```

In this script, we first generate simulated data using `np.random.rand()` and `np.random.randint()`. The input data `X` is a 2-dimensional array of shape (100, 2), and the target labels `y` are binary values.

Next, we define the model architecture using the `Sequen

# Prompts

In [12]:
prompt = PromptTemplate(
    input_variables = ['topic'],
    template = """Please tell me a joke about {topic} with 50 words."""
)
response = llm(prompt.format(topic="Irish"))
print(response.content)

Why don't you ever iron an Irish flag?

Because you don't want to press your luck!


# Chain

In [13]:
chain_one = prompt | llm | StrOutputParser()
response1 = chain_one.invoke({"topic": "Irish"})
print(response1)

Why don't Irish people play hide and seek?

Because someone always spots them at the pub!


In [14]:
prompt_second = PromptTemplate(
    input_variables = ['joke'],
    template = """Is this {joke} funny?"""
)
overall_chain = {"joke": chain_one} | prompt_second | llm | StrOutputParser()
response2 = overall_chain.invoke({"topic": "Irish"})
print(response2)

Yes, it is a punny joke! It plays on the words "Dublin" (the capital city of Ireland) and "doubling" the tempo in music.


# Embedding & Vector

In [15]:
prompt3 = PromptTemplate(
    input_variables = ["story"],
    template="""please make up a beautiful story based on topic of {story} within 300 words"""
)

story_example = llm(prompt3.format(story = 'princess'))
print(story_example.content)

Once upon a time, in a faraway kingdom, there lived a kind-hearted princess named Aurora. She had a heart as pure as gold and a smile that could light up the darkest of rooms. Aurora was loved by her people and admired for her compassion towards everyone, from the humblest of villagers to the grandest of nobles.

One day, a terrible curse fell upon the kingdom. A wicked sorceress named Morgana cast a spell that plunged the land into eternal darkness. Desperate to save her people, Princess Aurora embarked on a perilous journey to find the legendary Crystal of Light, the only object powerful enough to break the curse.

With unwavering determination, Aurora traveled through treacherous forests and crossed vast oceans, facing countless obstacles along the way. She encountered mythical creatures, who became her loyal companions, and wise old sages, who guided her with their wisdom. Despite the challenges, Aurora never lost hope.

Finally, after months of searching, Aurora reached the Cave o

In [16]:
#text splitter
test_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0
)

texts = test_splitter.create_documents([story_example.content])
texts[0].page_content

'Once upon a time, in a faraway kingdom, there lived a kind-hearted princess named Aurora. She had a'

In [17]:
#embedding
embeddings = OpenAIEmbeddings(
    openai_api_base="https://api.xiaoai.plus/v1",
    openai_api_key= os.getenv('OPENAI_API_KEY'),
)
query_result = embeddings.embed_query(texts[0].page_content)
print(query_result[:3])

[0.011465082, -0.03161949, -0.014830855]


In [18]:
#pinecone
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = 'langchain-quickstart-today'

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
vector_store.add_documents(documents=texts)

['5540bd3b-05a4-428b-8a37-687770bc626d',
 '2bfd2a37-3191-4387-b0b9-56f152ed0b0e',
 '1ac533af-ddc7-4e88-a0a2-abc08f9559cd',
 '74c0d8fd-53f5-489c-9d74-68eb0ffc9056',
 '468bfc84-a736-4740-809a-a9559477e6bc',
 '18a0a2fd-0305-4dc5-9a80-5ba410b41d20',
 '650ac681-7730-4b6f-b611-114b222671d2',
 'f94139b1-42b3-4fb6-bb71-86a868dcc8ee',
 '5ce49393-7170-4f10-b559-a5b4ea4a0f4b',
 '7472c46d-3809-4883-a717-53d6a8f75b52',
 'd9abc928-1c7b-4b61-8cae-89b8d447fa02',
 '0edc670d-e5db-4daf-afe3-94e386ec73f0',
 '7b892ab6-8643-4fb9-8664-1c459635c0ec',
 'bdffbbef-d5c7-4e1f-9be2-5c388ae0a791',
 'ef75e919-fb34-42a0-9fa4-25c854858141',
 '64a005cd-e4d7-4b9a-9ea9-48e42ce1c215',
 '6b53acf0-6b9b-4288-abc4-8015a9260071',
 '35021b53-6553-495e-81f4-2a8b580e41cc',
 '8a6ce28b-7c18-4cda-9bdd-0c934af4ea5e',
 'f676a968-a71c-4067-ac99-b9d8b7cd2acd',
 'fe818de2-3ed2-4615-a8e2-5c9bf3eb9244',
 '9bce7c45-b27e-48bf-bcdb-dd92a04265cd',
 '360ef897-f4ee-401f-8655-37b25103ef24',
 '6dd14fbb-0121-44c7-9dda-76bcce01d080',
 '4cd64d7f-c5bb-

In [19]:
query = "what happend in that kingdom?"
results = vector_store.similarity_search_with_score(query=query, k=2)
for res in results:
    print(res)

(Document(page_content='peace and prosperity to the kingdom.'), 0.848613918)
(Document(page_content='the cave, and the curse was lifted from the kingdom. The land was once again bathed in sunshine,'), 0.834380031)


# Agents